In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import calendar
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from wordcloud import WordCloud, STOPWORDS
import folium
from folium import plugins
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd

# 1. Merge datasets

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934 611(M) 2017-2018.dsv'
data2018=pd.read_csv(data_location,delimiter="|")

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/NUSE 934-611-611M ENERO2019.csv'
data2019=pd.read_csv(data_location,delimiter=";")

In [ ]:
frames = [data2018, data2019]
data = pd.concat(frames)
merged_nuse = data.loc[data['TIPO_DETALLE'] == '934 - RIÑA']
merged_nuse.reset_index(inplace=True)
merged_nuse.drop(columns=['index'], inplace=True)

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv',index=None)

# 2. Rebuild missing data

In [ ]:
localidadCodDictionaryNuse = {1:'USAQUEN',
                              2:'CHAPINERO',
                              3:'SANTA FE',
                              4:'SAN CRISTOBAL',
                              5:'USME',
                              6:'TUNJUELITO',
                              7:'BOSA',
                              8:'KENNEDY',
                              9:'FONTIBON',
                              10:'ENGATIVA',
                              11:'SUBA',
                              12:'BARRIOS UNIDOS',
                              13:'TEUSAQUILLO',
                              14:'LOS MARTIRES',
                              15:'ANTONIO NARIÑO',
                              16:'PUENTE ARANDA',
                              17:'CANDELARIA',
                              18:'RAFAEL URIBE URIBE',
                              19:'CIUDAD BOLIVAR',
                              20:'SUMAPAZ',
                              99:'SIN LOCALIZACION'}

## Methods to rebuild

In [ ]:
import import_ipynb
import ws_address
from selenium.common.exceptions import TimeoutException
import re
import unidecode

In [ ]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [ ]:
tags = ["Dirección ingresada: ","Dirección encontrada: ","Tipo dirección: ","Código postal: ","Sector catastral: ",
        "UPZ: ","Localidad: ","Latitud: ","Longitud: ","CHIP: "]
def parse_address_ws(ws_result):
    location = {}
    for idx in range(len(tags)-1):
        location[tags[idx].replace(': ','')] = find_between(ws_result,tags[idx],tags[idx+1])
    return location

In [ ]:
def assign_upz(original_df,index,UPZ_ws_field):
    original_df.at[index,'COD_UPZ'] = find_between(UPZ_ws_field, '(', ')')
    original_df.at[index,'UPZ'] = find_between(UPZ_ws_field, '', ' (')

In [ ]:
def get_cod_localidad(localidad_name):
    return [key  for (key, value) in localidadCodDictionaryNuse.items() if value == localidad_name][0]

In [ ]:
def rebuild_location_in_nuse(original_df, index, driver):
    address = original_df.at[index,'STR_DIRECCION_INCIDENTE']
    print(address)
    result_ws = ws_address.web_scrap_address(driver,address)
    ws_address.delete_address(driver,address)
    print(result_ws)

    if result_ws != "Not found":
        parsed_result = parse_address_ws(result_ws)
        print(parsed_result)
        original_df.at[index,'LATITUD'] = float(parsed_result['Latitud'])
        original_df.at[index,'LONGITUD'] = float(parsed_result['Longitud'])
        parsed_localidad = parsed_result['Localidad']
        if parsed_localidad == 'ANTONIO NARIÑO':
            original_df.at[index,'LOCALIDAD'] = parsed_localidad
        else:
            original_df.at[index,'LOCALIDAD'] = unidecode.unidecode(parsed_localidad)
        original_df.at[index,'COD_LOCALIDAD'] = int(get_cod_localidad(original_df.at[index,'LOCALIDAD']))
        original_df.at[index,'SEC_CATASTRAL'] = parsed_result['Sector catastral']
        assign_upz(original_df,index,parsed_result['UPZ'])
        return "Rebuilt"
    else:
        return "Not processed"


In [ ]:
def rebuild_address_in_nuse(original_df, index):
    log_text = original_df.at[index,'LOG_TEXT']
    address_found = re.search(address_regex,log_text)

    if address_found != None:
        parsed_address = clean_address(address_found)
        print(parsed_address.strip())
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = parsed_address.strip()
        return "Rebuilt"
    else:
        original_df.at[index,'STR_DIRECCION_INCIDENTE'] = 'ND'
        return "Not processed"

def clean_address(address_found):
    exclude_char_list = ['~','/','*','(',')']
    one_occurrence = address_found.group().split(',,,')[0].replace(',',' ')
    final_address = one_occurrence
    
    for char in exclude_char_list:
        if char in one_occurrence:
            final_address = final_address.split(char)[0]
            
    numbers_in_substring = re.sub('[^0-9]','', final_address)
    numbers_proportion = len(numbers_in_substring)/len(final_address)
    
    if numbers_proportion < 0.2:
        final_address = 'ND'
    
    return final_address


## Implement rebuild methods

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/merged_nuse.csv'
merged_nuse=pd.read_csv(data_location,delimiter=",")

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor ''":(merged_nuse == '').sum().values,
              "Celdas con valor ' '":(merged_nuse == ' ').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

### Rebuild address through log_text

In [ ]:
#Try to rebuild missing address through log_text field
df_empty_locations_without_address = merged_nuse.loc[merged_nuse['STR_DIRECCION_INCIDENTE'] == '-']
list_idx_rebuild_address = list(df_empty_locations_without_address.index.values)

In [ ]:
len(list_idx_rebuild_address)

In [ ]:
address_regex= '(CL|DG|KR|TV)+\s\d+.*(,,)'
registers_to_process = len(list_idx_rebuild_address)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild_address:
    state = rebuild_address_in_nuse(merged_nuse, index)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

In [ ]:
merged_nuse.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv',index=None)

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor 'ND'":(merged_nuse == 'ND').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

### Rebuild location through address

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [ ]:
df1 = df_input.loc[df_input['COD_UPZ'] == '-']
df2 = df_input.loc[df_input['UPZ'] == '-']
df3 = df_input.loc[df_input['COD_SEC_CATAST'] == '-']
df4 = df_input.loc[df_input['SEC_CATASTRAL'] == '-']
df5 = df_input.loc[df_input['COD_BARRIO'] == '-']
df6 = df_input.loc[df_input['BARRIO'] == '-']

In [ ]:
df1.equals(df2) and df1.equals(df3) and df1.equals(df4) and df1.equals(df5) and df1.equals(df6)

In [ ]:
#Try to rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' through address
df_empty_locations_with_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] != 'ND']
list_idx_rebuild_location = list(df_empty_locations_with_address.index.values)
len(list_idx_rebuild_location)

In [ ]:
#Rebuild 'sector catastral', 'UPZ', 'localidad', 'latitud', 'longitud' using web scraping
df_output = df_input
url='https://mapas.bogota.gov.co'
driver = ws_address.web_scrap_page(url)
registers_to_process = len(list_idx_rebuild_location)
rebuilt_registers = 0
registers_not_processed = 0
other_condition_counter = 0

for index in list_idx_rebuild_location:
    state = rebuild_location_in_nuse(df_output, index, driver)
    
    if state == "Rebuilt":
        rebuilt_registers += 1
    elif state == "Not processed":
        registers_not_processed += 1
    else:
        other_condition_counter += 1
    
    print('Rebuilt registers: ',rebuilt_registers,'/',registers_to_process)
    print('Registers not processed: ',registers_not_processed, '/', registers_to_process)

In [ ]:
print(rebuilt_registers)
print(registers_not_processed)
print(other_condition_counter)

In [ ]:
df_output.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_locations_nuse_19112019.csv',index=None)

In [ ]:
pd.DataFrame({"Tipo de dato":merged_nuse.dtypes.values,
              "Celdas con valor '-'":(merged_nuse == '-').sum().values,
              "Celdas con valor 'ND'":(merged_nuse == 'ND').sum().values,
              "Celdas vacías": merged_nuse.isna().sum().values},
             index=merged_nuse.columns)

In [ ]:
#Registers without address can not be rebuilt
df_empty_locations_without_address = df1.loc[df1['STR_DIRECCION_INCIDENTE'] == 'ND']
list_idx_not_rebuild = list(df_empty_locations_without_address.index.values)
len(list_idx_not_rebuild)

In [ ]:
# TODO: assign ND to df_empty_locations_without_address on location fields

## Verify registers are inside Bogotá

In [ ]:
def get_loc_from_coor(lat,lon):
    
    point=Point(lon,lat)
    for loc, geo in loc_.values:
        if point.within(geo[0]):
            return loc
    return "No Encontrado"

def get_upz_from_coor(lat,lon):

    point=Point(lon,lat)

    for cod,upz, geo in upz_.values:
        if point.within(geo):
            return cod,upz
    return "No Encontrado"

def get_loc_first(permutaciones):

    for p in permutaciones:
        localidad=get_loc_from_coor(p[0],p[1])
        if localidad != 'No Encontrado':
            break
    return localidad,p[0],p[1]

def get_loc_from_shape(lat,lon):
    
    ly=lat
    lx=lon
    permutaciones=[(ly , lx),(-ly , lx),(-ly, -lx),(ly , -lx),(lx , ly),(-lx , ly),(-lx, -ly),(lx , -ly)]
    return list(get_loc_first(permutaciones))

def recover_coors(df,col_lat_y,col_lon_x):
    coors=df[[col_lat_y,col_lon_x]].values
    new_data=[]
    for i in coors:
        row=get_loc_from_shape(i[0],i[1])
        __UPZ__=get_upz_from_coor(row[1],row[2])
        if len(__UPZ__) > 2:
            row+=[__UPZ__]*2
        else:
            row+=__UPZ__
        new_data.append(row)
    new_data=pd.DataFrame(new_data,columns=["LOCALIDAD_from_coors",'new_'+col_lat_y,'new_'+col_lon_x,'cod_UPZ','upz_from_coors'])

    return df.join(new_data)


In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv'
data = pd.read_csv(data_location,delimiter=",")

In [ ]:
loc_geo="/Users/anamaria/Desktop/dev/security_project/assets/loca.geojson.json"
loc_=gpd.read_file(loc_geo)
loc_=loc_[["LocNombre",'geometry']]

upz_geo="/Users/anamaria/Desktop/dev/security_project/assets/upla.json"
upz_=gpd.read_file(upz_geo)
upz_=upz_[["UPlCodigo",'UPlNombre','geometry']]

In [ ]:
col_lat='LATITUD'
col_lon='LONGITUD'
col_localidad='LOCALIDAD'
col_direccion='STR_DIRECCION_INCIDENTE'
cod_loc='COD_LOCALIDAD'
col_upz='UPZ'
cod_upz='COD_UPZ'
col_barrio='BARRIO'

Ubicacion=data[[col_lat,col_lon,col_localidad]]
new_Ubicacion=recover_coors(Ubicacion,col_lat,col_lon)

index_no_loc_out_coor=data[(data[col_direccion] == '-') & (data[cod_loc] == 99) & (new_Ubicacion.LOCALIDAD_from_coors == 'No Encontrado')].index
index_perm=data[(new_Ubicacion.LOCALIDAD_from_coors != 'No Encontrado') ].index

LOC_COD=data[[col_localidad,cod_loc]].drop_duplicates()
LOC_COD_={}

for i in LOC_COD.values:
    LOC_COD_[i[0]]=i[1]

for i in index_perm:
    data.at[i,col_localidad] = new_Ubicacion.loc[i]['LOCALIDAD_from_coors']
    data.at[i,cod_loc] = LOC_COD_[new_Ubicacion.loc[i]['LOCALIDAD_from_coors']]
    data.at[i,col_lon] = new_Ubicacion.loc[i]['new_'+col_lon]
    data.at[i,col_lat] = new_Ubicacion.loc[i]['new_'+col_lat]
    data.at[i,col_upz] = new_Ubicacion.loc[i]['upz_from_coors']
    data.at[i,cod_upz] = new_Ubicacion.loc[i]['cod_UPZ']

index_input_from_dir=data[(data[cod_loc] != 99) &  (data[col_direccion] != '-') & (new_Ubicacion.LOCALIDAD_from_coors == 'No Encontrado') ].index

  
index_no_dir_coor_out=data[(data[cod_loc] != 99) &  (data[col_direccion] == '-') & (new_Ubicacion.LOCALIDAD_from_coors != data[col_localidad])].index


data=data.drop(index_no_loc_out_coor)
data=data.drop(index_no_dir_coor_out)

In [ ]:
new_Ubicacion

In [ ]:
data.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/verify_coor_bog_nuse_25112019.csv',index=None)

In [ ]:
data.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/new_ubicacion_nuse_25112019.csv',index=None)

# 3. Standardise

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/rebuild_address_nuse_18112019.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [ ]:
#create timpestamp col to handle time ranges on unique event process
df_input['time_stamp']=pd.to_datetime(df_input['FECHA'] + ' ' + df_input["HORA"].astype(str).str.rjust(4,'0'))

In [ ]:
pd.DataFrame({"Tipo de dato":df_input.dtypes.values,
              "Celdas con valor '-'":(df_input == '-').sum().values,
              "Celdas con valor 'ND'":(df_input == 'ND').sum().values,
              "Celdas vacías": df_input.isna().sum().values},
             index=df_input.columns)

## 3.1 One register per event: event that occurs within 400 mts radius and 20 minutes time interval

### Find duplicated events

In [ ]:
import time, datetime
time_offset = 20
coor_offset = 0.001

In [ ]:
def find_duplicated_events(df, row):
    current_time = row['time_stamp']
    current_lat = row['LATITUD']
    current_lon = row['LONGITUD']
    current_point=Point(current_lon,current_lat)

    duplicated_event_idx = {}
    limit_time_interval = current_time + datetime.timedelta(minutes = time_offset)
    df_event_time = df.loc[(df['time_stamp'] >= current_time) & (df['time_stamp'] < limit_time_interval)]
    
    lat_point_list = [current_lat-coor_offset, current_lat-coor_offset, current_lat+coor_offset, current_lat+coor_offset]
    lon_point_list = [current_lon+coor_offset, current_lon-coor_offset, current_lon-coor_offset, current_lon+coor_offset]
    polygon_event = Polygon(zip(lon_point_list, lat_point_list))
    
    for index, row in df_event_time.iterrows():
        point=Point(row['LONGITUD'],row['LATITUD'])
        if point.within(polygon_event):
            #duplicated_event_idx.append(index)
            duplicated_event_idx[index] = row['STR_NUMERO_INTERNO']
    return duplicated_event_idx

In [ ]:
df_output = df_input.copy()

In [ ]:
df_output['dup_event'] = df_output.apply (lambda row: find_duplicated_events(df_output, row), axis=1)

In [ ]:
df_output.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/standardise_find_dup_event_nuse_26112019.csv',index=None)

### Delete duplicated events: preserve the first event on dup_event column

In [ ]:
data_location = '/Users/anamaria/Desktop/dev/security_project/datasets/standardise_find_dup_event_nuse_26112019.csv'
df_input = pd.read_csv(data_location,delimiter=",")

In [ ]:
pd.DataFrame({"Tipo de dato":df_input.dtypes.values,
              "Celdas con valor '-'":(df_input == '-').sum().values,
              "Celdas con valor 'ND'":(df_input == 'ND').sum().values,
              "Celdas vacías": df_input.isna().sum().values},
             index=df_input.columns)
print(df_input.shape)

In [ ]:
#Get index of registers that should be deleted
import ast
df = df_input
list_idx_repeated = []
list_idx_preserved = []
registers_to_process = len(df)
list_idx_processed =[]
counter_processed = 0

for index, row in df.iterrows():
    dup_event_x = ast.literal_eval(df.at[index,'dup_event'])
    current_dup_events = list(dup_event_x.keys())

    if (current_dup_events[0] not in list_idx_processed) & (current_dup_events[0] not in list_idx_preserved):
        list_idx_preserved.append(current_dup_events[0])
        list_idx_processed.append(current_dup_events[0])
        current_dup_events.pop(0)

    for idx_event in current_dup_events:
        if idx_event not in list_idx_processed:
            list_idx_repeated.append(idx_event)
            list_idx_processed.append(idx_event)
                
    counter_processed += 1
    
    print('Registers processed: ',counter_processed,'/',registers_to_process)


In [ ]:
#check (quantitatively) ID of preserved and repeated events index was succesful
print(len(list_idx_repeated)+len(list_idx_preserved))
print(len(list_idx_processed))
join_list = list_idx_preserved + list_idx_repeated

import collections
seen = set()
uniq = []
for x in join_list:
    if x not in seen:
        uniq.append(x)
        seen.add(x)

print(len(uniq))

lst = join_list
dupItems = []
uniqItems = {}
for x in lst:
    if x not in uniqItems:
        uniqItems[x] = 1
    else:
        if uniqItems[x] == 1:
            dupItems.append(x)
        uniqItems[x] += 1
        
print(len(dupItems))

In [ ]:
df_output = df_input.copy()

In [ ]:
df_output=df_output.drop(list_idx_repeated)
df_output.drop(columns=['dup_event','time_stamp'],inplace=True)
df_output.reset_index(inplace=True)

In [ ]:
df_output.to_csv(r'/Users/anamaria/Desktop/dev/security_project/datasets/standardise_result_nuse_27112019.csv',index=None)

In [ ]:
#save lists
MyFile=open('/Users/anamaria/Desktop/dev/security_project/datasets/list_idx_preserved_27112019.txt','w')
MyList=map(lambda x: str(x)+'\n', list_idx_preserved)
MyFile.writelines(MyList)
MyFile.close()

In [ ]:
#save lists
MyFile=open('/Users/anamaria/Desktop/dev/security_project/datasets/list_idx_repeated_27112019.txt','w')
MyList=map(lambda x: str(x)+'\n', list_idx_repeated)
MyFile.writelines(MyList)
MyFile.close()